# Land-use forcing

Smith et al. 2020:

- 2014 - 1850 land use forcing of -0.18 W m-2

The time series that scales with AFOLU is our best bet.

In [ ]:
from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties
from scipy.optimize import root_scalar

import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import pooch

In [ ]:
climate_response_df = pd.read_csv('../data/fair-calibrations/4xCO2_energy_balance_ebm3.csv')

In [ ]:
target_forcing = -0.18

scenarios = ['ssp245']
configs = ['UKESM1-0-LL']
species = ['CO2 AFOLU', 'Land use']

In [ ]:
def erf_rootfinder(x):
    f = FAIR()
    f.define_time(1850, 2015, 1)
    f.define_scenarios(scenarios)
    f.define_configs(configs)
    f.define_species(*read_properties(species=species))
    f.allocate()
    f.fill_from_rcmip()
    fill(f.climate_configs['ocean_heat_capacity'], climate_response_df.loc[0, 'C1':'C3'])
    fill(f.climate_configs['ocean_heat_transfer'], climate_response_df.loc[0, 'kappa1':'kappa3'])
    fill(f.climate_configs['deep_ocean_efficacy'], climate_response_df.loc[0, 'epsilon'])
    fill(f.climate_configs['gamma_autocorrelation'], climate_response_df.loc[0, 'gamma'])
    fill(f.species_configs['forcing_scale'], x, specie='Land use')
    initialise(f.forcing, 0)
    initialise(f.temperature, 0)
    initialise(f.cumulative_emissions, 0)  # necessary
    f.fill_species_configs()
    f.run(progress=False)
    print(f.forcing[-1, 0, 0, 1], x)
    return f.forcing[-1, 0, 0, 1] - target_forcing

In [ ]:
root_scalar(erf_rootfinder, 1)

In [ ]:
# pl.plot(f.timebounds, f.forcing_sum[:,0,0])
# pl.plot(f.timebounds, f.forcing_sum[:,0,0])

In [ ]:
# pl.plot(f.timebounds, f.forcing.loc[dict(specie='Ozone', scenario='ssp245', config='UKESM1-0-LL')])
# pl.plot(o3_ukesm.index.map(float), o3_ukesm)